---
title: "Analisi di visibilità"
description: "Un’analisi di visibilità consente di stabilire quali sono le porzioni di paesaggio visibili da un osservatore posto in un determinato luogo ed ad una determinata quota."
lang: it
date: "2022 04 07"
open-graph:
    title: "Analisi di visibilità"
    description: "Un’analisi di visibilità consente di stabilire quali sono le porzioni di paesaggio visibili da un osservatore posto in un determinato luogo ed ad una determinata quota."
    image: ../../project_logo.jpg
categories: [point, raster]
image: ../../project_logo.jpg
format:
    html:
        toc: true
        code-fold: false
execute:
  enabled: false
---

L’analisi di visibilità è un argomento molto interessante da trattare con Python. In un [mio articolo](https://massimilianomoraca.it/blog/gis/analisi-di-visibilita/) l’ho affrontata usando QGIS, qui oggi vediamo come raggiungere lo stesso obiettivo con Python.

## Prima di iniziare


### Librerie
Per raggiungere l'obiettivo userò le librerie che seguono:


In [1]:
from pathlib import Path
import geopandas as gpd
import rioxarray as rxr
import rasterio as rio
from rasterio.plot import show
import matplotlib.pyplot as plt
from xrspatial import hillshade, viewshed
from xarray.plot import imshow

### Fonti dati
Come [DEM](https://massimilianomoraca.it/blog/gis/3d-open-source/) ho usato un ritaglio del progetto [DEM20 di ISPRA](http://www.sinanet.isprambiente.it/it/sia-ispra/download-mais/dem20/view) su cui ho posizionato con QGIS undici punti; nell'articolo del blog uso l'undicesimo, qui proverò ad usarli tutti.

In [2]:
sample_data = Path.cwd().parent.parent.parent.joinpath('sample_data/viewshed')
dem = sample_data.joinpath('ispra_dtm_20.tif')
points = sample_data.joinpath('poi.shp')

output_folder = Path.cwd().parent.parent.parent.joinpath('output_folder')
Path.mkdir(output_folder, exist_ok=True)

FileExistsError: [Errno 17] File exists: '/media/max/Windows11/DEV/MIO/PyGIS_blog/output_folder'

# 1. Lettura dei dati

In [ ]:
points_data = gpd.read_file(points)
points_data

In [ ]:
raster = rxr.open_rasterio(dem).squeeze()
raster

Il vettore dei punti è molto semplice, ha una colonna con le geometrie ed una che identifica il singolo punto; quest'ultima informazione la userò più avanti. Il DEM è rettangolare e monobanda ed in effetti mi aspetto che sia monobanda poichè è quella che contiene le informazioni altimetriche in questo caso, con `squeeze()` estraggo la banda.

Il DEM ha come valore minimo di quota 0 metri, ma quale è la quota massima?

In [ ]:
max_h = int(raster.max().data)
print(f'La quota massima è {max_h} metri')

## 1.1 Visualizzazione dei dati grezzi

In [ ]:
raster_data = rio.open(dem)

fig, ax = plt.subplots(figsize=(10, 10))
show(
    source=raster_data,
    cmap='tab20c_r',
    contour=True,
    ax=ax,
    transform=raster_data.transform
)
show(
    source=raster_data,
    cmap='terrain_r',
    ax=ax,
    transform=raster_data.transform
)
for _index, row in points_data.iterrows():
    coordinates = row.geometry.xy
    ax.scatter(
        *coordinates,
        s=1000,
        marker="*",
        facecolor='red',
        edgecolor='black'
    )
    plt.text(
        x=coordinates[0][0],
        y=coordinates[1][0],
        s=row.fid,
        fontdict=dict(color='black', size=10),
        bbox=dict(facecolor='white', alpha=0.75)
    )
plt.show()

# 2. Analisi su un singolo punto

In [ ]:
single_point = points_data[points_data['fid'] == 11]
single_point

Ho selezionato solo il punto con id 11, è quello che ho usato nell'articolo del mio blog. Prima di partire con l'analisi è bene sapere che il risultato di una analisi di visibilità, detta anche *viewshed*, è un raster con valori che vanno da 0 a 180. Questi sono gradi e fanno riferimento alla visuale verticale dal punto di osservazione. Un valore pari a 0° sta ad indicare che quel pixel è proprio sotto il punto di osservazione, un valore pari a 90° sta a significare che il pixel è sull'orizzonte del punto di osservazione ed un valore pari a 180°, di conseguenza, indica che il pixel è proprio sopra il punto di osservazione.

In [ ]:
make_hillshade = hillshade(raster)

fig, ax = plt.subplots(figsize=(20, 10))
imshow(
    make_hillshade,
    ax=ax,
    cmap='gist_gray',
)
imshow(
    raster,
    ax=ax,
    cmap='terrain',
    vmin=1,
    vmax=max_h,
    levels=100,
    alpha=0.75
)
for _index, row in single_point.iterrows():
    coordinates = row.geometry.xy
    ax.scatter(
        *coordinates,
        s=1000,
        marker="*",
        facecolor='red',
        edgecolor='black'
    )
    plt.text(
        x=coordinates[0][0],
        y=coordinates[1][0],
        s=row.fid,
        fontdict=dict(color='black', size=10),
        bbox=dict(facecolor='white', alpha=0.75)
    )

plt.show()

Per una migliore resa visiva ho inserito la hillshade sotto al DEM tematizzato. Questa volta anzicchè usare la funzione `show` di *rasterio* ho preferito usare `imshow` di *xarray* per sperimentarla un po' visto che non l'avevo mai usata. E' interessante il fatto che aggiunga automaticamente la `colorbar` per ogni raster, devo capire come fare in modo che non venga visualizzata sempre e come disporla in altro modo.

## 2.1 Quota dell'osservatore

La quota dell'osservatore è un dato fondamentale per portare a termine una analisi di visibilità. Può essere la quota del punto in cui si trova l'osservatore o una quota fittizia compatibile con la topografia dell'area(l'osservatore fittizio non può essere a 100 metri di altezza se stiamo esaminando un'area in cui la quota minima è 500 metri). Quello che manca nei miei dati è proprio la quota!

Ma non è un problema perchè mi viene in soccorso [un'esperianza](https://maxdragonheart.github.io/PyGIS-Blog/point/2022/03/31/da-point-a-pointz.html) già fatta di recente e che vado a riprodurre:

In [ ]:
x_coord = single_point.iloc[0].geometry.xy[0][0]
y_coord = single_point.iloc[0].geometry.xy[1][0]

dataarray_value = raster.sel(x=x_coord, y=y_coord, method="nearest")
pixel_value = dataarray_value.data

pixel_value

Ora che ho tutto quello che mi serve posso effettuare l'analisi di visibilità.

In [ ]:
view = viewshed(raster, x=x_coord, y=y_coord, observer_elev=pixel_value)
view

In [ ]:
view_sel = view.where(view >= 0)
view_sel

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
imshow(view_sel, ax=ax)
plt.show()

Analisi effettuata, ora sono curioso di vedere quali aree ricadono nel campo visivo dell'osservatore.

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
imshow(
    make_hillshade,
    ax=ax,
    cmap='gist_gray',
)
imshow(
    raster,
    ax=ax,
    cmap='terrain',
    vmin=1,
    vmax=max_h,
    levels=100,
    alpha=0.75
)
imshow(view_sel, ax=ax, cmap='Wistia')


ax.scatter(
    *single_point.iloc[0].geometry.xy,
    s=1000,
    marker="*",
    facecolor='red',
    edgecolor='black'
)
plt.text(
    x=coordinates[0][0],
    y=coordinates[1][0],
    s=row.fid,
    fontdict=dict(color='black', size=10),
    bbox=dict(facecolor='white', alpha=0.75)
)

plt.show()

Le aree in tonalità di giallo sono quelle che vede l'osservatore, in questo caso vede benissimo Positano, gli isolotti del Gallo e di Vetara ma anche parte del Golfo di Napoli, forse anche il Vesuvio.

# 3. Analisi multipla

E' arrivato il momento di usare tutti i punti!

In [ ]:
for _index, row in points_data.iterrows():
    point_geometry = row.geometry.xy
    x_coord = point_geometry[0][0]
    y_coord = point_geometry[1][0]

    dataarray_value = raster.sel(x=x_coord, y=y_coord, method="nearest")
    pixel_value = dataarray_value.data

    view = viewshed(raster, x=x_coord, y=y_coord, observer_elev=pixel_value)

    view_sel = view.where(view >= 0)
    # Salvataggio dei risultati
    output_file = output_folder.joinpath(f'viewshed_point{row.fid}_h{pixel_value}.tif')
    view_sel.rio.to_raster(output_file)

![Point 1 - quota 875 metri](point_1.jpg "Point 1 - quota 875 metri")

![Point 2 - quota 413 metri](point_2.jpg "Point 2 - quota 413 metri")

![Point 3 - quota 106 metri](point_3.jpg "Point 3 - quota 106 metri")

![Point 4 - quota 0 metri](point_4.jpg "Point 4 - quota 0 metri")

![Point 5 - quota 0 metri](point_5.jpg "Point 5 - quota 0 metri")

![Point 6 - quota 475 metri](point_6.jpg "Point 6 - quota 475 metri")

![Point 7 - quota 0 metri](point_7.jpg "Point 7 - quota 0 metri")

![Point 8 - quota 1225 metri](point_8.jpg "Point 8 - quota 1225 metri")

![Point 9 - quota 0 metri](point_9.jpg "Point 9 - quota 0 metri")

![Point 10 - quota 0 metri](point_10.jpg "Point 10 - quota 0 metri")

![Point 11 - quota 450 metri](point_11.jpg "Point 11 - quota 450 metri")

# Conclusione

L'analisi di visibilità è fortemente infuenzata dalla risoluzione del DEM, più è spinta maggiore sarà la qualità dell'output. Di contro più è spinta la risoluzione del DEM maggiore sarà il tempo necessario per il processamento di ogni singolo punto di osservazione. Nel mio caso ci ho impiegato in media 4 secondi a punto.